In [1]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

import os
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm import tqdm
import math
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


# Setup

In [2]:
x_train = np.load("../input/ndscloaded/x_train.npy")
x_test = np.load("../input/ndscloaded/x_test.npy")
y_train = np.load("../input/ndscloaded/y_train.npy")
features = np.load("../input/ndscloaded/features.npy")
test_features = np.load("../input/ndscloaded/test_features.npy")
word_index = np.load("../input/ndscloaded/word_index.npy").item()
embedding_matrix = np.load("../input/ndscloaded/embedding_matrix.npy")

In [3]:
print(y_train)

[ 2 15 18 ... 25 25  7]


In [4]:
print(np.shape(embedding_matrix[25]))

(300,)


In [5]:
batch_size = 128

indices = np.arange(len(x_train)-1)
np.random.shuffle(indices)
i = 0
x_train_batch_tokens = np.take(x_train, indices[i*batch_size:(i+1)*batch_size], axis = 0) 
x_train_batch = [[embedding_matrix[token] for token in title] for title in x_train_batch_tokens]
y_train_batch_targets = np.take(y_train, indices[i*batch_size:(i+1)*batch_size])
y_train_batch = np.zeros((batch_size, 58))
y_train_batch[np.arange(batch_size), y_train_batch_targets] = 1

print(indices)
print(x_train_batch_tokens)
print(np.shape(x_train_batch))
print(y_train_batch_targets)
print(y_train_batch)

print(type(indices))
print(type(x_train_batch_tokens))
print(type(np.shape(x_train_batch[:1])))
print(type(x_train_batch))
print(type(y_train_batch_targets))
print(type(y_train_batch))

[292218 643469 395291 ... 267995 155923 437129]
[[    0     0     0 ...   103   242   262]
 [    0     0     0 ...   187 28889   133]
 [    0     0     0 ...    64    67    47]
 ...
 [    0     0     0 ...    40   200   436]
 [    0     0     0 ...    95   252  3797]
 [    0     0     0 ...    47    46   222]]
(128, 70, 300)
[35  3 41  4 22 34  3 18 27 22 37 29 37  3 32 32  1 31 34  3  5 18 18 35
 12  3 12 20 12 31  3 18  3 24 26  3  2  5 18 19 29  3 43  1 34  5  3 18
  5 18  3  5  7  3 32 31 26 18 41 30  3 12 18  3  4 35  4 21  3  5  3 31
 19  3 34  3 18 30  1  3 32 32 35 32  7 29  5  5 26 25 20 33  1 20  3  2
 49 42  5 27 31 20 26 47 16  3  3  3 32 35 45 18  5 47 28  4  5  3 31 18
 20 18 35  3 43 18 20 31]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'tuple'>
<class 'list'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>

In [6]:
# Data providers
batch_size = 128

indices = np.arange(len(x_train)-1)
# print(len(x_train))
# print(np.random.shuffle())

def batch_gen():
    n_batches = math.floor(len(x_train) / batch_size)
    while True: 
        np.random.shuffle(indices)  # Shuffle the data.
#         print(indices)
        for i in range(n_batches):
#             texts = train_df.iloc[i*batch_size:(i+1)*batch_size, 1]
#             text_arr = np.array([text_to_array(text) for text in texts])
            
            x_train_batch_tokens = np.take(x_train, indices[i*batch_size:(i+1)*batch_size], axis = 0) 
            x_train_batch = [[embedding_matrix[token] for token in title] for title in x_train_batch_tokens]
            y_train_batch_targets = np.take(y_train, indices[i*batch_size:(i+1)*batch_size])
            y_train_batch = np.zeros((batch_size, 58))
            y_train_batch[np.arange(batch_size), y_train_batch_targets] = 1

            yield (np.array(x_train_batch), y_train_batch)

# Training

In [7]:
from keras.models import Sequential
from keras.layers import CuDNNLSTM, Dense, Bidirectional, Activation

In [8]:
model = Sequential()
model.add(Bidirectional(CuDNNLSTM(64, return_sequences=True),
                        input_shape=(70, 300)))
model.add(Bidirectional(CuDNNLSTM(64)))
model.add(Dense(58))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [9]:
mg = batch_gen()
model.fit_generator(mg, epochs=2,
                    steps_per_epoch=1000,
                    verbose=True)

Epoch 1/2
1000/1000 [==============================] - 97s 97ms/step - loss: 1.8862 - acc: 0.4307
Epoch 2/2
1000/1000 [==============================] - 92s 92ms/step - loss: 1.2517 - acc: 0.6219


# Inference

In [ ]:
# prediction part
batch_size = 128
def batch_gen():
    n_batches = math.ceil(len(x_test) / batch_size)
    for i in range(n_batches):
        x_test_batch_tokens = x_test[i*batch_size:(i+1)*batch_size]
        x_test_batch = [[embedding_matrix[token] for token in title] for title in x_test_batch_tokens]
        yield np.array(x_test_batch)

import time
start_time = time.time()
all_preds = []
for x in batch_gen():
    all_preds.extend(model.predict(x))
print(time.time()-start_time)

In [ ]:
print(np.shape(all_preds))
y_te = [np.argmax(pred) for pred in all_preds]

test_df = pd.read_csv("../input/ndsc-beginner/test.csv")
submit_df = pd.DataFrame({"itemid": test_df["itemid"], "Category": y_te})
submit_df.to_csv("submission.csv", index=False)

In [ ]:
submit_df